In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
df = pd.read_csv(r'/home/ahmed/Ai/Machine learning from scratch/week2/Data.csv')
df

,0,1,2
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1
...,...,...,...
95,83.489163,48.380286,1
96,42.261701,87.103851,1
97,99.315009,68.775409,1
98,55.340018,64.931938,1


In [67]:
features, tests = df[['0', '1']], df['2']
x_train, x_test, y_train, y_test = train_test_split(features, tests, test_size=.3,random_state=1)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train) 
y_test = np.array(y_test)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((70, 2), (30, 2), (70,), (30,))

In [68]:
print("First five elements in X_train are:\n", x_train[:5])
print("Type of X_train:",type(x_train))

First five elements in X_train are:
 [[49.07256322 51.88321182]
 [74.775893   89.5298129 ]
 [50.28649612 49.80453881]
 [83.48916274 48.3802858 ]
 [78.63542435 96.64742717]]
Type of X_train: <class 'numpy.ndarray'>


### 1 Sigmoid function :

logistic regression model is represented as

$$ f_{\mathbf{w},b}(x) = g(\mathbf{w}\cdot \mathbf{x} + b)$$

function $g$ is the sigmoid function. The sigmoid function is defined as:

$$g(z) = \frac{1}{1+e^{-z}}$$


### 2 Cost function

logistic regression, the cost function is of the form 

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

where
* m is the number of training examples in the dataset
* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ is the cost for a single data point, which is - 

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$

### 3 gradient descent

the gradient descent algorithm is:

$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & b := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline       \; & w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1}\newline & \rbrace\end{align*}$$

where, parameters $b$, $w_j$ are all updated simultaniously
$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$
* m is the number of training examples in the dataset


In [69]:
class logistic_regression:
    def __init__(self, X, Y, W, B, alpha, iter):
        self.x = X
        self.y = Y
        self.w = W
        self.b = B  
        self.alpha = alpha
        self.iter = iter
        self.j_cost = []
        self.parameters = []
        return
    
    def sigmoid(self, z):
        g_z = 1 / (1 + np.exp(-z))
        return g_z        
        
    # this fucntion calcualte w * x + b
    def Z(self, i):
        z = np.dot(self.x[i], self.w) + self.b
        return z

    def loss(self, f_wb, i):
        Loss = -self.y[i] * np.log(f_wb) - (1 - self.y[i]) * np.log(1 - f_wb)
        return Loss
    
    def Cost_fucntion(self):
        
        # number of samples
        n = self.x.shape[0]
        cost = 0
        for i in range(n):
            
            z = self.Z(i)
            f_wb = self.sigmoid(z)

            cost += self.loss(f_wb, i)
            
        j_wb = cost / n

        return j_wb
        
        
    def derivative(self): # fitting
        
        n, m = self.x.shape
        d_dw = np.zeros(self.w.shape)
        d_db = 0        

        for i in range(n):
            z = self.Z(i)
            f_wb = self.sigmoid(z)            
            
            for j in range(m):
                d_dw[j] = d_dw[j] + (f_wb - self.y[i]) * self.x[i][j]         
            
            d_db = d_db + (f_wb - self.y[i])
        d_dw = d_dw / n
        d_db = d_db / n

        return d_dw, d_db


    def fit(self): # gradient_descent
        
        j_cost = []
        parameters = []
        
        for i in range(self.iter):
            d_dw, d_db = self.derivative()

            w_temp = self.w - self.alpha * d_dw
            b_temp = self.b - self.alpha * d_db
            
            cost_now = self.Cost_fucntion()
            self.j_cost.append(cost_now)
            self.parameters.append([self.w, self.b])
            self.w = w_temp
            self.b = b_temp
            if(i % 100 == 0):
                print(f"Iteration {i:4}: Cost {float(self.j_cost[-1]):8.2f}   when w = {self.w} and b = {self.b}")

    def predict(self, x_test):
        prediction = []
        n = len(x_test)
        for i in range(n):
            z = self.Z(i)
            f_wb = self.sigmoid(z)
            if f_wb > .5:
                prediction.append(1)
            else:    
                prediction.append(0)
        return prediction
    
    def accurece_score(self, x_prediction, y_test):
        
        total_ture = 0
        for i in range(len(y_test)):
            if x_prediction[i] == y_test[i]:
                total_ture += 1
        precentage = total_ture / len(y_test) * 100
        return precentage


    def coef_(self):
        return self.w
    def intercet_(self):
        return self.b
    def cost_per_parameters(self):
        for i in range(len(self.j_cost)):
            print(f"Iteration {i:2}: Cost {float(self.j_cost[i]):8.2f}   when w = {self.parameters[i][0]} and b = {self.parameters[i][1]}")


In [70]:
n = x_train.shape[1]

log_reg = logistic_regression(x_train, y_train, np.zeros(n), 0, 0, 0)
cost = log_reg.Cost_fucntion()
print(cost)

0.6931471805599454


## test

In [71]:
w = np.array([0.2, 0.2])
b = -24.
model = logistic_regression(x_test, y_test, w, b,0,0)
cost_mode = model.Cost_fucntion()
cost_mode

0.35869305283563985

# Fitting

In [72]:
m, n = x_train.shape
w_inti = 0.01 * (np.random.rand(2).reshape(-1,1) - 0.5)
b_inti = -8
alpha = .001
Iteration = 500
logistic = logistic_regression(x_train, y_train, w_inti, b_inti, alpha, Iteration)

In [73]:
logistic.fit()

Iteration    0: Cost     4.70   when w = [[0.04847268]
 [0.05271695]] and b = [-7.99937197]
Iteration  100: Cost     0.27   when w = [[0.07436183]
 [0.06178906]] and b = [-8.00110107]


/tmp/ipykernel_24764/3365210996.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Iteration {i:4}: Cost {float(self.j_cost[-1]):8.2f}   when w = {self.w} and b = {self.b}")


Iteration  200: Cost     0.27   when w = [[0.07442351]
 [0.06176335]] and b = [-8.00307037]
Iteration  300: Cost     0.27   when w = [[0.07443973]
 [0.06177835]] and b = [-8.00503908]
Iteration  400: Cost     0.27   when w = [[0.07445573]
 [0.06179352]] and b = [-8.00700715]


In [74]:
logistic.coef_()

array([[0.07447157],
       [0.06180855]])

In [75]:
print(logistic.Cost_fucntion())

[0.26990015]


In [76]:
pred = logistic.predict(x_train)
pred
print(len(pred), ' ', len(x_train))

70   70


In [77]:
print('model accurece score = ', logistic.accurece_score(pred, y_train))

model accurece score =  94.28571428571428
